In [1]:
import os
import json
import fasttext
from tqdm import tqdm
import json
# from transformers import AutoTokenizer

In [2]:
model = fasttext.load_model("/data/hyxu/codes/lowMT_compute/model/lid.176.ftz")
model

In [3]:
model.predict("sadnasjhfn iab asina winawfh siefnwvjawoi saf qwhoiasoidfw9 f as jfwoias;lk ")

(('__label__nl',), array([0.38591853]))

In [4]:

data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"
def _read(p):
    with open(p) as f:
        data = f.readlines()
    return [s.strip("\n") for s in data]
nl_data = _read(os.path.join(data_path, "train.nl"))
zh_data = _read(os.path.join(data_path, "train.zh"))

In [5]:
lang1, lang2 = "nl", "zh"
data_socre = {lang1: [], lang2: []}
r = 0.4
nl_data_result =[]
for s in tqdm(nl_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang1}" and result[1] > r:
        nl_data_result.append((s, result[1].item()))
zh_data_result =[]
for s in tqdm(zh_data):
    result = model.predict(s)
    if result[0][0] == f"__label__{lang2}" and result[1] > r:
        zh_data_result.append((s, result[1].item()))

100%|██████████| 8000000/8000000 [02:11<00:00, 60879.49it/s]


In [6]:
len(nl_data_result), len(zh_data_result)

(1961790, 6871402)

In [15]:
nl_data_result = [(d[0], d[1].item()) for d in nl_data_result]

In [16]:
with open(os.path.join(data_path, "clean.train.nl"), "w") as f:
    json.dump(nl_data_result, f, indent=4)
with open(os.path.join(data_path, "clean.train.zh"), "w") as f:
    json.dump(zh_data_result, f, indent=4)

## 写比赛结果

In [3]:
import os
import json
from transformers import MBartForConditionalGeneration, AutoTokenizer
from datasets import load_from_disk, DatasetDict, Dataset
import  xml.dom.minidom
from eval import generate

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
path = "/data/hyxu/codes/lowMT_compute/data/public_data/test"
data_path = "/data/hyxu/codes/lowMT_compute/data/public_data/train/mono"

In [2]:
def _read_xml(p):
    dom = xml.dom.minidom.parse(p)
    root = dom.documentElement
    seg_list = root.getElementsByTagName("seg")
    data = [s.firstChild.data for s in seg_list]
    print(len(data))
    return [s.rstrip("\n") for s in data]
data = _read_xml(os.path.join(path, "test.nl-zh.nl.xml"))
data

1000


['Bij een bezoek aan de plaatselijke school waren ze geschokt door de chaotische scenes die vaak in de klas voorkwamen.',
 'Ik zal in de volgende wedstrijd zeker meer aandacht besteden aan het passeren en de offensieve verandering.',
 'Denemarken, Zweden en Finland zijn de eerste landen ter wereld die warmtepomptechnologie gebruiken om stedelijk afvalwater te behandelen.',
 'De actualiteit van deze twee processen moet worden gegarandeerd.',
 'Ik kan niet anders zeggen dan dat ik heel trots ben.',
 'Met onze ruim 400 winkels zitten we echt overal in Nederland.',
 'De Philips Airfryer is de enige Airfryer met Fat Removal-technologie',
 'Ook zegt het college dat men wil streven naar minder inhuur.',
 'De beelden worden vervolgens beoordeeld met kunstmatige intelligentie.',
 'Bij opsporingsberichtgeving dient het OM strikt de Aanwijzing Opsporingsberichtgeving na te leven.',
 'Vandaag nodigen we 66e afgestudeerden van Tsinghua Universiteit uit om het verhaal van hun graduatiedistributie te

In [3]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = model.cuda()
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [4]:
predictions = generate(model, tokenizer, "nl_XX", "zh_CN", data, 16, 5, 128)

  0%|          | 0/63 [00:00<?, ?it/s]/data/hyxu/codes/nmt-corrector/transformers-4.28.1/src/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 63/63 [02:04<00:00,  1.98s/it]


In [6]:
dom = xml.dom.minidom.parse(os.path.join(path, "test.nl-zh.zh.xml"))
root = dom.documentElement
seg_list = root.getElementsByTagName("seg")
for s, p in zip(seg_list, predictions):
    s.firstChild.data = p
with open(os.path.join(path, "test.nl-zh.zh1.xml"), "w") as f:
    dom.writexml(f)

## 用tokenizer 清洗数据

In [6]:
model_path  = "/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer

MBartTokenizerFast(name_or_path='/data/hyxu/codes/lowMT_compute/model/ft-clean-smo/chpk-8000/model', vocab_size=250027, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN']}, clean_up_tokenization_spaces=True)

In [ ]:
with open(os.path.join(data_path, "clean.train.nl")) as f:
    nl_data_result = json.load(f)
with open(os.path.join(data_path, "clean.train.zh")) as f:
    zh_data_result = json.load(f)
len(nl_data_result), len(zh_data_result)

In [30]:
r = 15              # 拼接阈值
class CleanData(object):
    r =15
    clean_data = []
    noise_data = []
    def add(self, s):
        l = len(s)
        if l > r:
            self.clean_data.append(s)
        else:
            if len(self.noise_data) < 1:
                self.noise_data.append(s)
            else:
                new_string = self.noise_data.pop()
                new_string += s
                self.add(new_string)
nl_clean_data = CleanData()        
zh_clean_data = CleanData()        

In [31]:
tokenizer.tokenize("你是个大事必")

['▁你', '是个', '大事', '必']

In [32]:
from tqdm import  tqdm
for s in tqdm(nl_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    nl_clean_data.add(tokenized_text)

  1%|          | 14289/1961790 [00:22<08:24, 3858.20it/s]

In [ ]:
for s in tqdm(zh_data_result):
    tokenized_text = tokenizer.tokenize(s[0])
    zh_clean_data.add(tokenized_text)

In [ ]:
len(nl_clean_data)